# [Intrcranial hemorrhage detection](http://kaggle.com/c/rsna-intracranial-hemorrhage-detection)

Intracranial hemorrhage, bleeding that occurs inside the cranium, is a serious health problem requiring rapid and often intensive medical treatment. For example, intracranial hemorrhages account for approximately 10% of strokes in the U.S., where stroke is the fifth-leading cause of death. Identifying the location and type of any hemorrhage present is a critical step in treating the patient.

Diagnosis requires an urgent procedure. When a patient shows acute neurological symptoms such as severe headache or loss of consciousness, highly trained specialists review medical images of the patient’s cranium to look for the presence, location and type of hemorrhage. The process is complicated and often time consuming.

The goal of this notebook is to build an algorithm to detect acute intracranial hemorrhage and [its subtypes](https://www.kaggle.com/c/rsna-intracranial-hemorrhage-detection/overview/hemorrhage-types).



# Step 00 -- Setting-up

In [0]:
training_mode = True
setup = True
colab_mode = True
verbose = True
kaggle_kernelMode = False
download_rawData = True

In [0]:
MODEL_ID = 'seresnext50' #'seresnext101' #'EfficientNet-UNet' 

EPOCHS = 5
BATCH_SIZE = 2
CLASS_SIZE = 4
IMG_HEIGHT = 256
IMG_WIDTH = 1600 if MODEL_ID == 'seresnext101' else 512
ORIG_IMG_HEIGHT = 256
ORIG_IMG_WIDTH = 1600
NUM_CHANNELS = 3

FILTER_IMG_HEIGHT = 256
FILTER_IMG_WIDTH = 256
FILTER_THRESHOLD = 0.5
FILTER_MODEL_ID = 'DenseNet-BC-201'

DATASET_ID = 'intracranial_hemorrhage_2019'
NOTEBOOK_ID = '{}_mask_{}'.format(DATASET_ID, MODEL_ID)
BACKBONE = MODEL_ID
HISTORY_FILE = 'history.json'

GOOGLE_DRIVE = '/content/drive/'
GOOGLE_DRIVE_DIR = '{}/My\ Drive/'.format(GOOGLE_DRIVE)

DATA_DIR = '/content/' if colab_mode else '../input/rsna-intracranial-hemorrhage-detection/' if kaggle_kernelMode else './'
OUTPUT_DIR = DATA_DIR if colab_mode else './' 

WEIGHTS_DIR = DATA_DIR
PRETRAINED_WEIGHTS_DIR = DATA_DIR
CLASSIFIER_MODEL_DIR = PRETRAINED_WEIGHTS_DIR 

TRAIN_DATA_FILENAME = '{}_stage_1_train.zip'.format(DATASET_ID)
TRAIN_DATA_LABEL_FILENAME = '{}_stage_1_train.csv.zip'.format(DATASET_ID)
SAMPLE_SUBMISSION_FILENAME = '{}_stage_1_sample_submission.csv.zip'.format(DATASET_ID)

TRAIN_DATA_DIR = '../input/rsna-intracranial-hemorrhage-detection/train_images' if kaggle_kernelMode else '{}/train/'.format(DATA_DIR)
TRAIN_DIR = '../tmp/train/' if kaggle_kernelMode else '/tmp/train/'
TEST_DATA_DIR = 'test_images' if kaggle_kernelMode else 'test'
TEST_DIR = '{}/{}/'.format(DATA_DIR,
                           TEST_DATA_DIR)

BACKBONE_WEIGHTS_FILENAME = 'efficientnet-b4_imagenet_1000_notop.h5'
BACKBONE_WEIGHTS = ('{}/{}'.format(BACKBONE_WEIGHTS_DIR,
                                   BACKBONE_WEIGHTS_FILENAME)) if kaggle_kernelMode else 'ImageNet'

PRETRAINED_WEIGHTS_FILENAME = '{}_mask_{}_weights.h5'.format(DATASET_ID,
                                                             MODEL_ID)
PRETRAINED_WEIGHTS = '{}/{}'.format(PRETRAINED_WEIGHTS_DIR,
                                    PRETRAINED_WEIGHTS_FILENAME)

CLASSIFIER_MODEL_FILENAME = '{}_missing_mask_predictions_{}_checkpoint.h5'.format(DATASET_ID,
                                                                                  FILTER_MODEL_ID) #'severstal_steel_defect_missing_mask_predictions_DesneNet-BC-169_checkpoint.h5'
CLASSIFIER_MODEL_WEIGHTS = '{}/{}'.format(CLASSIFIER_MODEL_DIR,
                                          CLASSIFIER_MODEL_FILENAME)

CHECKPOINT_FILENAME = '{}_checkpoint.h5'.format(NOTEBOOK_ID)
CHECKPOINT_FILE = '{}/{}'.format(PRETRAINED_WEIGHTS_DIR,
                                 CHECKPOINT_FILENAME)

SAMPLE_SUBMISSION_FILENAME = 'sample_submission.csv' if kaggle_kernelMode else '{}_sample_submission.csv'.format(DATASET_ID)
SUBMISSION_FILENAME = 'submission.csv' if kaggle_kernelMode else '{}_submission.csv'.format(NOTEBOOK_ID) 
SUBMISSION_FILE = '{}/{}'.format(DATA_DIR,
                                 SUBMISSION_FILENAME)

In [4]:
if setup and colab_mode and download_rawData:
  from google.colab import files
  files.upload()

Saving kaggle.json to kaggle.json


In [0]:
if setup and colab_mode and download_rawData:
  ! mkdir /root/.kaggle/
  ! mv /content/kaggle.json /root/.kaggle/

In [6]:
if setup and download_rawData and not kaggle_kernelMode:
  ! pip install --upgrade kaggle
  ! kaggle competitions download -c rsna-intracranial-hemorrhage-detection

     |████████████████████████████████| 61kB 2.3MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.6-cp36-none-any.whl size=72859 sha256=d752b7121b847842478ed81632b46429250f3c7acd43969f4ba8743cab4e7866
  Stored in directory: /root/.cache/pip/wheels/57/4e/e8/bb28d035162fb8f17f8ca5d42c3230e284c6aa565b42b72674
Successfully built kaggle
  Found existing installation: kaggle 1.5.5
    Uninstalling kaggle-1.5.5:
      Successfully uninstalled kaggle-1.5.5
100% 156G/156G [49:07<00:00, 24.1MB/s]
100% 156G/156G [49:07<00:00, 56.8MB/s]


In [7]:
if setup and colab_mode:
  from google.colab import drive
  drive.mount(GOOGLE_DRIVE)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [8]:
import time
if setup and download_rawData and not kaggle_kernelMode:
  print ('Loading raw data using Kaggle API into Google Drive...')
  start = time.time()
  ! mkdir /{TRAIN_DATA_DIR}
  ! mv /{DATA_DIR}/rsna-intracranial-hemorrhage-detection.zip {DATA_DIR}/{TRAIN_DATA_FILENAME}
  ! cp {DATA_DIR}/{TRAIN_DATA_FILENAME} /{GOOGLE_DRIVE_DIR}/ 
  ! cp /{DATA_DIR}/{CHECKPOINT_FILENAME} /{GOOGLE_DRIVE_DIR}/
  print ('Loaded data into Google Drive in: {} seconds ...'.format(round(time.time()-start),2))
elif setup and colab_mode:
  print ('Loading data and saved weights from Google drive ...')
  start = time.time()
  ! cp {GOOGLE_DRIVE_DIR}/{TRAIN_DATA_FILENAME} {DATA_DIR}/
  ! cp {GOOGLE_DRIVE_DIR}/{CHECKPOINT_FILENAME} {DATA_DIR}/
  ! cp {GOOGLE_DRIVE_DIR}/{CLASSIFIER_MODEL_FILENAME} {DATA_DIR}/
  print ('Loaded data and saved weights from Google drive ...')
  print ('Loaded data from Google Drive in: {} seconds ...'.format(round(time.time()-start),2))

Loading raw data using Kaggle API into Google Drive...
cp: error writing '//content/drive//My Drive/intrcarnial_hemorrhage_2019_stage_1_train.zip': No space left on device
cp: cannot stat '//content//intrcarnial_hemorrhage_2019_mask_seresnext50_checkpoint.h5': No such file or directory
Loaded data into Google Drive in: 1115 seconds ...


In [0]:
if setup:
  ! unzip -q {DATA_DIR}/{TRAIN_DATA_FILENAME} -d TRAIN_DATA_DIR